In [269]:
# import pandas as pd
# # return the wordnet object value corresponding to the POS tag
# import nltk
# from nltk.corpus import wordnet
# import string
# from nltk import pos_tag
# from nltk.corpus import stopwords
# from nltk.tokenize import WhitespaceTokenizer
# from nltk.stem import WordNetLemmatizer
# nltk.download('stopwords')
# nltk.download('wordnet')

# from nltk.sentiment.vader import SentimentIntensityAnalyzer

# from sklearn.feature_extraction.text import TfidfVectorizer
# import gensim
# from gensim.test.utils import common_texts
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# from sklearn import preprocessing

In [1]:
import pandas as pd

df = pd.read_csv("data_translated.csv")
# feedbacks = df['Verbatim Translated']
# ratings = df['Rating']

reviews_df = df[['Verbatim Translated', 'Rating', 'Created']]
# print(reviews_df)
reviews_df['is_bad_review'] = reviews_df['Rating'].apply(lambda x:1 if x<5 else 0)

# reviews_df = reviews_df[['Verbatim Translated', 'is_bad_review']]

#sample data
reviews_df = reviews_df.sample(frac=1, replace=False, random_state=42)
print(reviews_df)

                                    Verbatim Translated  Rating  \
511                              increase the promotion       3   
51                    special rate for special account        5   
1171     maintain a good manner of service to customers      10   
2282  personal loans for me to recommend to the whol...      10   
557   enable back ur security verification previousl...       0   
...                                                 ...     ...   
1095                    good customer service at branch       8   
1130  estatement my account not available why furthe...       6   
1294                improve the internet banking system       0   
860                                        breath ok no       8   
3174   hire more relationship managers i have had my...      10   

                    Created  is_bad_review  
511   10/16/2020 2:00:12 PM              1  
51     8/27/2020 4:51:27 PM              0  
1171   2/21/2021 6:16:24 PM              0  
2282   5/24/202

In [271]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

import string
import nltk
from nltk import pos_tag
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
import datetime as dt
from sklearn import preprocessing
from nltk.sentiment.vader import SentimentIntensityAnalyzer    
    
def clean_text(text):
    # lower text
    text = str(text).lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
reviews_df['Verbatim_Cleaned'] = reviews_df['Verbatim Translated'].apply(lambda x: clean_text(x))
# reviews_df.head()

sid = SentimentIntensityAnalyzer()

reviews_df['sentiments'] = reviews_df['Verbatim Translated'].apply(lambda x: sid.polarity_scores(str(x)))
# reviews_df.head()

reviews_df = pd.concat([reviews_df.drop(['sentiments'], axis=1), reviews_df['sentiments'].apply(pd.Series)], axis=1)
reviews_df.head()

# add number of char 
reviews_df['nb_chars'] = reviews_df['Verbatim Translated'].apply(lambda x:len(str(x)))

# add number of words column
reviews_df['nb_words'] = reviews_df['Verbatim Translated'].apply(lambda x:len(str(x).split(' ')))
reviews_df.head()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Tan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Verbatim Translated,Rating,Created,is_bad_review,Verbatim_Cleaned,neg,neu,pos,compound,nb_chars,nb_words
511,increase the promotion,3,10/16/2020 2:00:12 PM,1,increase promotion,0.000,0.465,0.535,0.3182,22,3
51,special rate for special account,5,8/27/2020 4:51:27 PM,0,special rate special account,0.000,0.357,0.643,0.6597,33,6
1171,maintain a good manner of service to customers,10,2/21/2021 6:16:24 PM,0,maintain good manner service customer,0.000,0.674,0.326,0.4404,46,8
2282,personal loans for me to recommend to the whol...,10,5/24/2021 7:37:47 PM,0,personal loan recommend whole family friend,0.000,0.641,0.359,0.6808,66,12
557,enable back ur security verification previousl...,0,10/16/2020 4:21:57 PM,1,enable back ur security verification previousl...,0.098,0.808,0.094,-0.0454,259,46


In [272]:
def normalize(df):
    x = df.values.reshape(-1,1) #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df

def compute_time_score(x,recent_timestamp):

    epoch=recent_timestamp.timestamp()/x.timestamp()
    return epoch

# clean text data
reviews_df['Created'] = reviews_df['Created'].astype('datetime64[ns]')

recent_timestamp = reviews_df['Created'].max()
print(recent_timestamp)
reviews_df["time_score"] = reviews_df["Created"].apply(lambda x: compute_time_score(x,recent_timestamp))

time_score = reviews_df["time_score"]

reviews_df["time_score"] = normalize(time_score).values
reviews_df["review_clean"] = reviews_df["Verbatim Translated"].apply(lambda x: clean_text(x))

reviews_df["normalised_rating"] = normalize(reviews_df["Rating"]).values
reviews_df["normalised_compound"] = normalize(reviews_df["compound"]).values

# compute urgency score
reviews_df["urgency_score"] = reviews_df["compound"] * reviews_df["time_score"]
reviews_df = reviews_df[["Created", "Verbatim Translated", "Verbatim_Cleaned", "compound", "time_score", "urgency_score","Rating", "nb_chars", "nb_words"]]
reviews_df = reviews_df.sort_values("urgency_score")
reviews_df.head()

2021-12-07 23:04:30


,Created,Verbatim Translated,Verbatim_Cleaned,compound,time_score,urgency_score,Rating,nb_chars,nb_words
323,2020-09-09 15:15:37,"Before this, I left my name and phone number f...",leave name phone number extension moratorium b...,-0.9089,0.641814,-0.583345,5,574,106
559,2020-10-16 19:02:47,i previously did my fd by going to the branch ...,previously fd go branch instead online due cir...,-0.9667,0.588142,-0.568557,6,984,194
180,2020-08-27 15:25:04,change the out source debt collections teamthe...,change source debt collection teamthe team cal...,-0.8416,0.660633,-0.555989,6,170,31
165,2020-08-28 16:43:10,i experienced a lot of difficulties for the la...,experienced lot difficulty last one year use r...,-0.8360,0.659105,-0.551012,7,520,92
218,2020-09-09 13:58:37,upgrade your internet banking very not user fr...,upgrade internet bank user friendly make numer...,-0.8162,0.641891,-0.523912,1,294,52


In [268]:
print(reviews_df)
reviews_df.to_csv("results.csv")

                Created                                Verbatim Translated  \
323 2020-09-09 15:15:37  Before this, I left my name and phone number f...   
559 2020-10-16 19:02:47  i previously did my fd by going to the branch ...   
180 2020-08-27 15:25:04  change the out source debt collections teamthe...   
165 2020-08-28 16:43:10  i experienced a lot of difficulties for the la...   
218 2020-09-09 13:58:37  upgrade your internet banking very not user fr...   
..                  ...                                                ...   
66  2020-08-28 10:44:46  offer more rewarding packages for placing more...   
380 2020-04-09 09:33:00  customer satisfaction on the quality of servic...   
90  2020-08-27 18:38:23  revamp your website and upgrade your app your ...   
293 2020-03-09 10:40:55  further improve the rhb internet banking appli...   
454 2020-02-09 21:58:38  improve on services and promptness in respondi...   

                                      Verbatim_Cleaned  compoun

In [111]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'white',
        max_words = 200,
        max_font_size = 40, 
        scale = 3,
        random_state = 42
    ).generate(str(data))

    fig = plt.figure(1, figsize = (20, 20))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()
    
# print wordcloud
show_wordcloud(reviews_df["review_clean"])

,Verbatim Translated,neg
2274,stop spamming people phone with your advertise...,0.507
1510,please stop call spamming and unsolicited sale...,0.363
414,lower lg personal loan rate,0.355
1805,interest rate lower or slightly lower than oth...,0.328
610,call center never resolve my problem it take a...,0.328
567,security question making the transfer more tro...,0.327
1071,lots of offers but the dates of offer misleadi...,0.308
1234,because he really understands my desperate sit...,0.299
2297,actually i want have a complaint concerning th...,0.287
2031,good cash rebate mechanism for rhb shell visa ...,0.273
